## Pneumonia Detection using PyTorch

###  Special thanks to Paul Mooney for Datasets on Chest-Xray images for Pneumonia !!

Here is the link to dataset. https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia#chest_xray.zip

### Header declarations and detect  CUDA 

In [1]:
import os
import torch
from torch import nn
import numpy as np

import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
device='cpu'

if train_on_gpu:
    print("Cuda is available")
    device='cuda'
else:
    print("CPU is available")



ModuleNotFoundError: No module named 'torch'

###  Load Images from root folder and pre-process them and create DataLoaders

Transforms
When you load in the data with ImageFolder, you'll need to define some transforms. For example, the images are different sizes but we'll need them to all be the same size for training. You can either resize them with transforms.Resize() or crop with transforms.CenterCrop(), transforms.RandomResizedCrop(), etc. We'll also need to convert the images to PyTorch tensors with transforms.ToTensor().

In [ ]:
# define training and test data directories
data_dir = 'pneumonia_images/'
train_dir = os.path.join(data_dir, 'train/')
test_dir = os.path.join(data_dir, 'test/')

#preprocess and convert to tensor
data_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

#load and transform data from given path
train_data = datasets.ImageFolder(train_dir, transform=data_transform)
test_data =  datasets.ImageFolder(test_dir,transform=data_transform)

#create DataLoaders for training set and test set
train_loader = torch.utils.data.Dataloader(train_data,batch_size=32,shuffle=True)
test_loader  = torch.utils.data.Dataloader(test_data,batch_size=32,shuffle=True)


### Visualize data from test and train loaders

In [ ]:
# Visualize some sample data

# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    ax.set_title(labels[idx])

### Create A CNN architecture 
defining a model from scratch. This may not be nearly good enough for detecting infection.But let's just give it a try. 
The architecture has 5 Convolution layers, each convolution layer uses RELU activation function, followed by maxpool that reduces the width and height of input by a factor of 2. 

Final layers are 2 fully connected Linear layers with droput of 0.2 and again uses relu activation. 

We can compute the spatial size of the output volume as a function of the input volume size (W), the kernel/filter size (F), the stride with which they are applied (S), and the amount of zero padding used (P) on the border. The correct formula for calculating how many neurons define the output_W is given by (W−F+2P)/S+1.

For example for a 7x7 input and a 3x3 filter with stride 1 and pad 0 we would get a 5x5 output. With stride 2 we would get a 3x3 output.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class XRayNet(nn.Module):
    def __init__(self):
        super(XRayNet, self).__init__()
        # convolutional layer (sees 224x224*1 image tensor)
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        # convolutional layer (sees 112*112*16 image tensor)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # convolutional layer (sees 56*56*32 image tensor)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # convolutional layer (sees 28*28*64 image tensor)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
        # convolutional layer (sees 14*14*128 image tensor)
        self.conv5 = nn.Conv2d(128, 256, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # linear layer (7 * 7 * 256)
        self.fc1=nn.Linear(7 * 7 * 256,200)
        # linear layer (7 * 7 * 256)
        self.fc2=nn.Linear(200,2)
        #dropout layer
        self.dropout=nn.Dropout(0.2)
        
    def forward(self,x):
        # add sequence of convolutional and max pooling layers
        x= self.pool(F.Relu(self.conv1(x)))
        x= self.pool(F.Relu(self.conv2(x)))
        x= self.pool(F.Relu(self.conv3(x)))
        x= self.pool(F.Relu(self.conv4(x)))
        x= self.pool(F.Relu(self.conv5(x)))
        # flatten image input
        x = x.view(-1, 256 * 7 * 7)
        
        # first fully connected layer
        x= self.dropout(x)
        x= F.Relu(self.fc1(x))
        # second fully connected layer
        x= self.dropout(x)
        x= F.Relu(self.fc2(x))
        return x
    
# create a complete CNN
model = XRayNet()
print(model)

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

### Specify Loss Function and Optimizer


In [ ]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.Adam(model.parameters,lr=0.01)#optim.SGD(model.parameters(), lr=0.01)

### Train your Network

In [ ]:
num_epochs=20

for epoch in range(1,num_epochs):
    train_loss=0.0
    for images,labels in train_loader:
        images,labels=images.cuda(),labels.cuda()
        # reset to calculate new weights
        optimizer.zero_grad()
        #forward pass
        output=model(images)
        # calculate the batch loss
        loss = criterion(output, labels)
         # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*images.size(0)
    # calculate average losses
    train_loss = train_loss/len(train_loader)
    print('For Epoch {} \t training loss is {:.6f}',epoch,train_loss)

### Test the Trained Network

In [ ]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval() # prep model for evaluation

for data, target in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(len(target)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
test_loss = test_loss/len(test_loader.sampler)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))
      

